In [ ]:
pip install classy

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from classy import Class

# --- 1. 描画用フィギュアの準備 ---
plt.ioff()
fig, ax = plt.subplots(figsize=(10, 5))
l_axis = np.arange(2, 2501)
line, = ax.plot(l_axis, np.zeros_like(l_axis), lw=2, color='darkblue')

ax.set_xscale('log')
ax.set_xlim(1, 2500)
ax.set_ylim(0, 7000)
ax.set_xlabel(r'Multipole moment $\ell$', fontsize=12)
ax.set_ylabel(r'$\ell(\ell+1)C_\ell^{TT} / 2\pi \ [\mu\mathrm{K}^2]$', fontsize=12)
ax.grid(True, which="both", alpha=0.3)
plt.ion()

# --- 2. 計算・更新関数 ---
def update_plot(h, omega_b, omega_cdm, n_s, A_s_1e9, tau, Om_k, Neff, Yp):
    cosmo = Class()
    try:
        cosmo.set({
            'output': 'tCl', 'lensing': 'no',
            'h': h, 'omega_b': omega_b, 'omega_cdm': omega_cdm,
            'n_s': n_s, 'A_s': A_s_1e9*1e-9, 'tau_reio': tau,
            'Omega_k': Om_k, 'N_ur': Neff, 'YHe': Yp,
            'l_max_scalars': 2500
        })
        cosmo.compute()
        cls = cosmo.raw_cl(2500)
        l = cls['ell'][2:]
        dl_tt = l*(l+1)*cls['tt'][2:]/(2*np.pi) * (2.7255e6)**2
        
        line.set_ydata(dl_tt)
        display(fig)
    except:
        pass
    finally:
        cosmo.struct_cleanup()
        cosmo.empty()

# --- 3. 妥当な範囲（Planck観測ベース）でのウィジェット生成 ---
def create_row(val, v_min, v_max, v_step, latex):
    lbl = widgets.HTML(value=f'<div style="width:200px; font-size:14px;">{latex}</div>')
    sld = widgets.FloatSlider(value=val, min=v_min, max=v_max, step=v_step, 
                              continuous_update=True, 
                              layout=widgets.Layout(width='350px'),
                              readout=True, readout_format='.4f')
    return sld, widgets.HBox([lbl, sld])

# 各パラメータを「妥当な範囲」に設定
# $h$: 0.6-0.8 (ハッブル緊張の議論を含む範囲)
s_h, ui_h = create_row(0.6736, 0.60, 0.80, 0.005, r"$h$")

# $\omega_b$: 0.0224付近 (非常に狭い範囲で効く)
s_ob, ui_ob = create_row(0.02237, 0.020, 0.025, 0.0001, r"$\omega_b = \Omega_b h^2$")

# $\omega_{cdm}$: 0.12付近
s_oc, ui_oc = create_row(0.120, 0.09, 0.15, 0.001, r"$\omega_{cdm} = \Omega_c h^2$")

# $n_s$: 0.965付近 (1.0を跨ぐかどうかは重要)
s_ns, ui_ns = create_row(0.9649, 0.90, 1.05, 0.005, r"$n_s$")

# $A_s$: 2.1e-9付近
s_as, ui_as = create_row(2.10, 1.5, 3.0, 0.05, r"$10^9 A_s$")

# $\tau$: 0.054付近 (0.01-0.10程度が現実的)
s_tau, ui_tau = create_row(0.0544, 0.03, 0.09, 0.002, r"$\tau_{reio}$")

# $\Omega_k$: ほぼ0だが、可視化のために±0.05
s_ok, ui_ok = create_row(0.0, -0.05, 0.05, 0.002, r"$\Omega_k$")

# $N_{eff}$: 標準3.044付近
s_ne, ui_ne = create_row(3.044, 2.0, 4.5, 0.05, r"$N_{eff}$")

# $Y_p$: 標準0.245付近
s_yp, ui_yp = create_row(0.245, 0.20, 0.30, 0.005, r"$Y_p$")

ui = widgets.VBox([ui_h, ui_ob, ui_oc, ui_ns, ui_as, ui_tau, ui_ok, ui_ne, ui_yp])
out = widgets.interactive_output(update_plot, {
    'h': s_h, 'omega_b': s_ob, 'omega_cdm': s_oc, 'n_s': s_ns, 
    'A_s_1e9': s_as, 'tau': s_tau, 'Om_k': s_ok, 'Neff': s_ne, 'Yp': s_yp
})

display(ui, out)

Output()